In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import mysql.connector

load_dotenv()  # This loads the .env file

# Fetch credentials from environment
mysql_user = os.getenv('MYSQL_USER')
mysql_password = os.getenv('MYSQL_PASSWORD')

def connect_to_sakila():
    connection = mysql.connector.connect(
        host='localhost',
        user=mysql_user,
        password=mysql_password,S
        database='sakila'
    )
    return connection

# Establish connection
try:
    conn = connect_to_sakila()
    print("Connection established successfully!")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    if 'conn' in locals() and conn.is_connected():
        conn.close()

Connection established successfully!


In [48]:
# Fetch credentials from environment
mysql_user = os.getenv('MYSQL_USER')
mysql_password = os.getenv('MYSQL_PASSWORD')
mysql_host = 'localhost'
mysql_db = 'sakila'

# Create the database connection engine
engine = create_engine(f'mysql+mysqlconnector://{mysql_user}:{mysql_password}@{mysql_host}/{mysql_db}')

def rentals_month(engine, month, year):
    """
    Fetch rental data for a given month and year from the Sakila database.

    Parameters:
    - engine: Database connection engine (SQLAlchemy engine).
    - month: Integer representing the month (1-12).
    - year: Integer representing the year (e.g., 2024).

    Returns:
    - Pandas DataFrame containing the rental data for the specified month and year.
    """
    # SQL query to retrieve rental data for the given month and year
    query = f"""
    SELECT rental_id, rental_date, inventory_id, customer_id, return_date, staff_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    
    # Execute the query and return the result as a DataFrame
    rental_data = pd.read_sql(query, engine)
    
    return rental_data

# Example usage
month = 5  # May
year = 2005  # Example year

# Get rental data for the specified month and year
rental_data = rentals_month(engine, month, year)

# Display the first few rows of the rental data
rental_data.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1


Let's check what years are available in the database and the rental table

In [49]:
def available_years(engine):
    """
    Retrieve the distinct years from the rental_date column in the rental table.
    
    Parameters:
    - engine: Database connection engine (SQLAlchemy engine).
    
    Returns:
    - Pandas DataFrame containing distinct years available in the rental table.
    """
    # SQL query to get distinct years from the rental_date column
    query = """
    SELECT DISTINCT YEAR(rental_date) AS rental_year
    FROM rental
    ORDER BY rental_year;
    """
    
    # Execute the query and return the result as a DataFrame
    years_data = pd.read_sql(query, engine)
    
    return years_data

# Example usage
years_data = available_years(engine)

# Display the distinct years
print(years_data)

   rental_year
0         2005
1         2006


In [50]:
def rental_count_month(rentals_df, month, year):
    """
    Count the number of rentals made by each customer during the specified month and year.
    
    Parameters:
    - rentals_df: DataFrame containing rental data for the month and year.
    - month: Integer representing the month (1-12).
    - year: Integer representing the year (e.g., 2005).
    
    Returns:
    - A new DataFrame containing the count of rentals for each customer in the specified month and year.
    """
    # Create the column name based on the month and year (e.g., "rentals_05_2005")
    column_name = f"rentals_{month:02d}_{year}"
    
    # Group by 'customer_id' and count the rentals (size of each group)
    rental_counts = rentals_df.groupby('customer_id').size().reset_index(name=column_name)
    
    return rental_counts

# Example usage for May 2005
month = 5  # May
year = 2005  # Example year

# Get rental data for May 2005 (using your rentals_month function)
rental_data = rentals_month(engine, month, year)

# Get rental count by customer for May 2005
rental_count_df = rental_count_month(rental_data, month, year)

# Display the rental counts DataFrame
print(rental_count_df)

     customer_id  rentals_05_2005
0              1                2
1              2                1
2              3                2
3              5                3
4              6                3
..           ...              ...
515          594                4
516          595                1
517          596                6
518          597                2
519          599                1

[520 rows x 2 columns]


In [52]:
def rentals_month(engine, month, year):
    """
    Retrieves rental data for a given month and year from the Sakila database.
    
    Parameters:
    - engine: The database connection engine.
    - month: Integer representing the month (1-12).
    - year: Integer representing the year (e.g., 2005).
    
    Returns:
    - A Pandas DataFrame containing the rental counts for the specified month and year.
    """
    # Build the SQL query to fetch rental data for the specified month and year
    query = f"""
    SELECT customer_id, COUNT(*) AS rentals
    FROM rental
    WHERE YEAR(rental_date) = {year} AND MONTH(rental_date) = {month}
    GROUP BY customer_id;
    """
    
    # Execute the query and fetch the results into a DataFrame
    rental_data = pd.read_sql(query, engine)
    
    # Create a column name based on the month and year (e.g., rentals_05_2005)
    column_name = f"rentals_{month:02d}_{year}"
    
    # Rename the 'rentals' column to match the dynamic name
    rental_data.rename(columns={'rentals': column_name}, inplace=True)
    
    return rental_data

# Example usage
month1, year1 = 5, 2005  # May 2005
month2, year2 = 6, 2005  # June 2005

# Get rental data for the two months (using the rentals_month function)
rental_data_may_2005 = rentals_month(engine, month1, year1)
rental_data_june_2005 = rentals_month(engine, month2, year2)

# Check the columns of both DataFrames
rental_data_may_2005.head()
rental_data_june_2005.head()

,customer_id,rentals_06_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
